In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import datasets, layers, models
import os
import tarfile
import scipy.io as sio
import numpy as np
import random
import cv2
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
def save_train_data(fnames, labels, bboxes):
    src_folder ='/tmp/car_ims'
    num_samples = len(fnames)

    train_split = 0.8
    num_train = int(round(num_samples * train_split))
    train_indexes = random.sample(range(num_samples), num_train)

    for i in range(num_samples):
        fname = fnames[i][8:]
        label = labels[i]
        (x1, y1, x2, y2) = bboxes[i]
        src_path = os.path.join(src_folder, fname)

        src_image = cv2.imread(src_path)
        height, width = src_image.shape[:2]

        # margins of 16 pixels (Crop image according to a given bounding box)
        margin = 16
        x1 = max(0, x1 - margin)
        y1 = max(0, y1 - margin)
        x2 = min(x2 + margin, width)
        y2 = min(y2 + margin, height)

        if i in train_indexes:
            dst_folder = '/tmp/car_ims/train/'
        else:
            dst_folder = '/tmp/car_ims/valid/'

        dst_path = os.path.join(dst_folder, label)
        if not os.path.exists(dst_path):
            os.makedirs(dst_path)
        dst_path = os.path.join(dst_path, fname)
        dst_image = src_image[y1:y2, x1:x2]
        cv2.imwrite(dst_path, dst_image)

### Download and Separate Data

We first need to download the data from a remote server and seperate the data into training and testing sets. Note that you only need to download the data once, if you have downloaded the data before do not run the code in this section.

In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/car_ims.tgz \
  -O /tmp/car_ims.tgz

--2020-08-14 14:26:45--  http://imagenet.stanford.edu/internal/car196/car_ims.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1956628579 (1.8G) [application/x-gzip]
Saving to: ‘/tmp/car_ims.tgz’

/tmp/car_ims.tgz     10%[=>                  ] 189.53M  33.1MB/s    eta 1m 43s ^C


In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/cars_annos.mat \
  -O /tmp/cars_annos.mat

--2020-08-14 14:26:58--  http://imagenet.stanford.edu/internal/car196/cars_annos.mat
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394471 (385K) [text/plain]
Saving to: ‘/tmp/cars_annos.mat’

/tmp/cars_annos.mat 100%[===================>] 385.23K   422KB/s    in 0.9s    

2020-08-14 14:26:59 (422 KB/s) - ‘/tmp/cars_annos.mat’ saved [394471/394471]



KeyboardInterrupt: ignored

In [ ]:
!wget --no-check-certificate \
  http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels.mat \
  -O /tmp/cars_test_annos.mat

In [ ]:
!rm -rf /tmp/car_ims/train/
!rm -rf /tmp/car_ims/test/
!rm -rf /tmp/car_ims/valid/

Unzip the compressed data. (You do not need to understand how this works)

In [ ]:
local_tar = '/tmp/car_ims.tgz'
my_tar = tarfile.open(local_tar)
my_tar.extractall('/tmp')
my_tar.close()

In [ ]:
car_annotation= sio.loadmat('/tmp/cars_annos.mat')

annotations = car_annotation['annotations']
annotations = np.transpose(annotations)

fnames = []
class_ids = []
labels = []
bboxes  = []
for annotation in annotations:
  bbox_x1 = annotation[0][1][0][0]
  bbox_y1 = annotation[0][2][0][0]
  bbox_x2 = annotation[0][3][0][0]
  bbox_y2 = annotation[0][4][0][0]

  class_id = annotation[0][5][0][0]
  bboxes.append((bbox_x1, bbox_y1, bbox_x2, bbox_y2))
  labels.append(f'{class_id:06}')

  file_name = annotation[0][0][0]
  class_ids.append(class_id)
  fnames.append(file_name)

labels_count = np.unique(class_ids).shape[0]
print(np.unique(class_ids))
print('The number of different cars is %d' % labels_count)

save_train_data(fnames, labels, bboxes)

### Create a data generator

In [ ]:
datagen = ImageDataGenerator()
train_iter = datagen.flow_from_directory('/tmp/car_ims/train/', class_mode='categorical', batch_size=32)
test_iter = datagen.flow_from_directory('/tmp/car_ims/valid/', class_mode='categorical', batch_size=32)

### Build Model

In [ ]:
# TODO
base_model = tf.keras.applications.ResNet101(
    include_top=False, weights='imagenet'
)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(196, activation='softmax')

inputs = tf.keras.Input(shape=(256, 256, 3))
x = base_model(inputs)
x = tf.keras.layers.Flatten()(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.Dense(300)(x)

outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()





Model: "functional_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_55 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet101 (Functional)       (None, None, None, 2048)  42658176  
_________________________________________________________________
flatten_16 (Flatten)         (None, 131072)            0         
_________________________________________________________________
dense_46 (Dense)             (None, 196)               25690308  
Total params: 68,348,484
Trainable params: 68,243,140
Non-trainable params: 105,344
_________________________________________________________________


In [ ]:
# Training
with tf.device("GPU:0"): 
  history = model.fit(train_iter, epochs=10, validation_data=test_iter)

Epoch 1/10
405/405 [==============================] - 178s 440ms/step - loss: 2.8355 - accuracy: 0.4595 - val_loss: 1.2247 - val_accuracy: 0.6923
Epoch 2/10
405/405 [==============================] - 177s 436ms/step - loss: 0.1107 - accuracy: 0.9827 - val_loss: 1.0476 - val_accuracy: 0.7386
Epoch 3/10
405/405 [==============================] - 177s 436ms/step - loss: 0.0485 - accuracy: 0.9947 - val_loss: 0.9603 - val_accuracy: 0.7680
Epoch 4/10
405/405 [==============================] - 176s 435ms/step - loss: 0.0280 - accuracy: 0.9963 - val_loss: 0.9311 - val_accuracy: 0.7766
Epoch 5/10
405/405 [==============================] - 176s 434ms/step - loss: 0.0148 - accuracy: 0.9972 - val_loss: 0.9552 - val_accuracy: 0.7751
Epoch 6/10
405/405 [==============================] - 176s 435ms/step - loss: 0.0147 - accuracy: 0.9975 - val_loss: 0.9784 - val_accuracy: 0.7745
Epoch 7/10
405/405 [==============================] - 177s 436ms/step - loss: 0.0141 - accuracy: 0.9972 - val_loss: 1.1237 -